**Taylor M. Smith**

**Assignment #4**

**4433 Data Visualization**

**WNTR 2023**

**Using free and open source tools, provide a set of choropleth visualizations for each of the columns containing dates such that the resulting visualizations (48 states only) tell the story** **by conveying through color, texture, or both the time lines of achievement of each milestone/column in the provided dataset. Missing data are of particular interest in that when a** **state has never achieved a given milestone, that should be indicated in a standout manner such as cross-hatching. Consider that the publication may be grayscale. Provide a solution for that** **as well**. **Provide the titles, labels, and legends necessary for clarification. File support is given as follows: SturmCodebook has the explanation. SturmData is the data CSV.**
***

In [2]:
# importing the required libraties 
import pandas as pd
import folium 
import webbrowser
from folium.plugins import StripePattern
import geopandas as gpd
import numpy as np
import json
import requests
from folium.features import DivIcon
from IPython.display import display

In [3]:
#importing csv and viewing first few columns of df
sturm = pd.read_csv('SturmData.csv')
sturm.head()

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN
1,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0
2,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0


In [4]:
# checking rows and types of data
sturm.info()
len(sturm)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          48 non-null     object 
 1   fips           48 non-null     int64  
 2   icpsr          48 non-null     int64  
 3   debtfree       47 non-null     float64
 4   effectivemwpa  47 non-null     float64
 5   earnings       44 non-null     float64
 6   wills          32 non-null     float64
 7   soletrader     31 non-null     float64
dtypes: float64(5), int64(2), object(1)
memory usage: 3.1+ KB


48

In [5]:
url = ('https://raw.githubusercontent.com/python-visualization/folium/master/examples/data')
state_dems = f"{url}/us-states.json"

# reading the file & printing it
geoJ = gpd.read_file(state_dems)
geoJ.head()


,id,name,geometry
0,AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [6]:
# creating a list to put the states in
geoJ_states = list(geoJ.id.values)
len(geoJ_states)

50

In [7]:
# renaming the column from 'id' to 'state' in the 'geoJ' to merge the two dfs.
geoJ_df = geoJ.rename(columns = {"id":"state"})

# then, we merge the sample 'strum' and the geoJ df on the 'id'.
updated_df = geoJ_df.merge(sturm, on = "state")


updated_df

,state,name,geometry,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1,41,1846.0,NaN,1887.0,NaN,NaN
1,AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",4,61,1864.0,1871.0,1973.0,NaN,1871.0
2,AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",5,42,1835.0,1873.0,1873.0,1868.0,1868.0
3,CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,Colorado,"POLYGON ((-107.91973 41.00391, -105.72895 40.9...",8,62,1861.0,1861.0,1861.0,1874.0,1874.0
5,CT,Connecticut,"POLYGON ((-73.05353 42.03905, -71.79931 42.022...",9,1,1845.0,1877.0,1877.0,1809.0,1877.0
6,DE,Delaware,"POLYGON ((-75.41409 39.80446, -75.50720 39.683...",10,11,1865.0,1873.0,1873.0,NaN,NaN
7,FL,Florida,"POLYGON ((-85.49714 30.99754, -85.00421 31.003...",12,43,1845.0,1943.0,1892.0,1823.0,NaN
8,GA,Georgia,"POLYGON ((-83.10919 35.00118, -83.32279 34.787...",13,44,1868.0,1873.0,1861.0,1867.0,NaN
9,ID,Idaho,"POLYGON ((-116.04751 49.00024, -116.04751 47.9...",16,63,1887.0,1903.0,1915.0,1887.0,1887.0


In [8]:
# addinng boundaries
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data"
)
us_states = f'{url}/us-states.json'
geoJ_dta = json.loads(requests.get(us_states).text)

In [13]:
# folium map 1
f_map = folium.Map(location=[39, -95], zoom_start=3)

folium.Choropleth(
geo_data = updated_df,
data = updated_df,
columns=['state', "wills"],
key_on = "feature.properties.state",
fill_color = 'YlGnBu',
fill_opacity = 1,
line_opacity = 0.2,
legend_name = "Wills Map",
smooth_factor = 0,
Highlight = True,
line_color = "#0000",
name = "wills",
show = False,
overlay = True,
nan_fill_color = "pink"
).add_to(f_map)


# adding dash line to make states pop
folium.GeoJson(
    geoJ_dta,
    style_function = lambda feature: {
        "fillColor": "white",
        "color": "purple",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(f_map)

# cross-hatching for null vals
nans = updated_df [updated_df["wills"].isnull()] ['state'].values
gdf_nans = updated_df [updated_df['state'].isin(nans)]
sp = StripePattern(angle = 45, color = 'pink', space_color = 'white')
sp.add_to(f_map)
folium.features.GeoJson(name = "Show Wills NaN values", data = gdf_nans, style_function = lambda x :{'fillPattern': sp},show = True).add_to(f_map)

# hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = updated_df,
    style_function = style_function, 
    control = False,
    highlight_function = highlight_function, 
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['state','wills'],
        aliases = ['state','wills'],
        style = ("background-color: white; color: #333333; font-family: arial; font-size: 13px; padding: 13px;") 
    )
)
f_map.add_child(NIL)
f_map.keep_in_front(NIL)

# map title
folium.map.Marker(
    [48, -102],
    icon=DivIcon(
        icon_size = (250,36),
        icon_anchor = (-120, 102),
        html='<div style="font-size: 20pt">Wills Map<div>',
        )
    ).add_to(f_map)

display(f_map)

f_map.save('wills.html')

In [14]:
# folium map 2
f2_map = folium.Map(location=[39, -95], zoom_start=4)

folium.Choropleth(
geo_data = updated_df,
data = updated_df,
columns = ["state", "debtfree"],
key_on = "feature.properties.state",
fill_color = 'PuBuGn',
fill_opacity = 1,
line_opacity = 0.2,
legend_name = "Debt Free",
smooth_factor = 0,
Highlight= True,
line_color = "#0000",
name = "Debt Free Map",
show = False,
overlay = True,
nan_fill_color = "White"
).add_to(f2_map)

# adding dash line to make states pop
folium.GeoJson(
    geoJ_dta,
    style_function = lambda feature: {
        "fillColor": "Red",
        "color": "white",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(f2_map)

# cross-hatching for null vals
nans = updated_df[updated_df["debtfree"].isnull()]['state'].values

gdf_nans = updated_df[updated_df['state'].isin(nans)]
sp = StripePattern(angle = 45, color='grey', space_color = 'gray')
sp.add_to(f2_map)
folium.features.GeoJson(name = "Click for debtfree NaN values", data = gdf_nans, style_function = lambda x :{'fillPattern': sp},show = True).add_to(f2_map)



# hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = updated_df,
    style_function = style_function, 
    control = False,
    highlight_function = highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['state','debtfree'],
        aliases=['state','debtfree'],
        style=("background-color: gray; color: #333333; font-family: arial; font-size: 13px; padding: 13px;") 
    )
)
f2_map.add_child(NIL)
f2_map



# map title
folium.map.Marker(
    [48, -102],
    icon = DivIcon(
        icon_size = (250,36),
        icon_anchor = (-120, 102),
        html='<div style="font-size: 20pt">Debt Free Map<div>',
        )
    ).add_to(f2_map)

display(f2_map)

f2_map.save('debtfree.html')


In [15]:
# folium map 3
f3_map = folium.Map(location = [39, -95], zoom_start = 3)

folium.Choropleth(
geo_data = updated_df,
data = updated_df,
columns = ["state", "effectivemwpa"],
key_on = "feature.properties.state",
fill_color = 'PuBuGn',
fill_opacity = 1,
line_opacity = 0.2,
legend_name = "MWPA",
smooth_factor = 0,
Highlight = True,
line_color = "#0000",
name = "Effective MWPA Map",
show = False,
overlay = True,
nan_fill_color = "Pink"
).add_to(f3_map)

# adding dash line to make states pop
folium.GeoJson(
    geoJ_dta,
    style_function = lambda feature: {
        "fillColor": "pink",
        "color": "blue",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(f3_map)

# cross-hatching for null vals
nans = updated_df[updated_df["effectivemwpa"].isnull()]['state'].values

gdf_nans = updated_df[updated_df['state'].isin(nans)]
sp = StripePattern(angle = 45, color = 'grey', space_color = 'grey')
sp.add_to(f3_map)
folium.features.GeoJson(name = "show effectivemwpa NaN values", data = gdf_nans, style_function = lambda x :{'fillPattern': sp},show = True).add_to(f3_map)



# hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = updated_df,
    style_function = style_function, 
    control = False,
    highlight_function = highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['state','effectivemwpa'],
        aliases=['state','effectivemwpa'],
        style=("background-color: grey; color: #333333; font-family: arial; font-size: 13px; padding: 13px;") 
    )
)
f3_map.add_child(NIL)
f3_map



# map title
folium.map.Marker(
    [48, -102],
    icon = DivIcon(
        icon_size = (250,36),
        icon_anchor = (-120, 102),
        html='<div style="font-size: 20pt">Effective Mwpa Map<div>',
        )
    ).add_to(f3_map)

display(f3_map)

f3_map.save('effectivemwpa.html')


In [16]:
# folium map 4
f4_map = folium.Map(location = [39, -95], zoom_start = 3)

folium.Choropleth(
geo_data = updated_df,
data = updated_df,
columns = ["state", "earnings"],
key_on = "feature.properties.state",
fill_color = 'PuBuGn',
fill_opacity = 1,
line_opacity = 0.2,
legend_name = "Earnings",
smooth_factor = 0,
Highlight = True,
line_color = "#0000",
name = "Earnings Map",
show = False,
overlay = True,
nan_fill_color = "Blue"
).add_to(f4_map)

# adding dash line to make states pop
folium.GeoJson(
    geoJ_dta,
    style_function = lambda feature: {
        "fillColor": "red",
        "color": "blue",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(f4_map)

# cross-hatching for null vals
nans = updated_df[updated_df["earnings"].isnull()]['state'].values

gdf_nans = updated_df[updated_df['state'].isin(nans)]
sp = StripePattern(angle = 45, color = 'grey', space_color = 'grey')
sp.add_to(f4_map)
folium.features.GeoJson(name = "show earnings NaN values", data = gdf_nans, style_function = lambda x :{'fillPattern': sp},show = True).add_to(f4_map)



# hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = updated_df,
    style_function = style_function, 
    control = False,
    highlight_function = highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['state','earnings'],
        aliases=['state','earnings'],
        style=("background-color: grey; color: #333333; font-family: arial; font-size: 13px; padding: 13px;") 
    )
)
f4_map.add_child(NIL)
f4_map


# map title
folium.map.Marker(
    [48, -102],
    icon = DivIcon(
        icon_size = (250,36),
        icon_anchor = (-120, 102),
        html='<div style="font-size: 20pt">Earnings Map<div>',
        )
    ).add_to(f4_map)

display(f4_map)

f4_map.save('earnings.html')


In [17]:
# folium map 5
f5_map = folium.Map(location = [39, -95], zoom_start = 3)

folium.Choropleth(
geo_data = updated_df,
data = updated_df,
columns = ["state", "soletrader"],
key_on = "feature.properties.state",
fill_color = 'PuBuGn',
fill_opacity = 1,
line_opacity = 0.2,
legend_name = "Sole Trader",
smooth_factor = 0,
Highlight = True,
line_color = "#0000",
name = "Sole Trader Map",
show = False,
overlay = True,
nan_fill_color = "Green"
).add_to(f5_map)

# adding dash line to make states pop
folium.GeoJson(
    geoJ_dta,
    style_function = lambda feature: {
        "fillColor": "white",
        "color": "grey",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(f5_map)

# cross-hatching for null vals
nans = updated_df[updated_df["soletrader"].isnull()]['state'].values

gdf_nans = updated_df[updated_df['state'].isin(nans)]
sp = StripePattern(angle = 45, color = 'grey', space_color = 'grey')
sp.add_to(f5_map)
folium.features.GeoJson(name = "show sole trader NaN values", data = gdf_nans, style_function = lambda x :{'fillPattern': sp},show = True).add_to(f5_map)



# hover functionality
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = updated_df,
    style_function = style_function, 
    control = False,
    highlight_function = highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['state','soletrader'],
        aliases=['state','soletrader'],
        style=("background-color: grey; color: #333333; font-family: arial; font-size: 13px; padding: 13px;") 
    )
)
f5_map.add_child(NIL)
f5_map


# map title
folium.map.Marker(
    [48, -102],
    icon = DivIcon(
        icon_size = (250,36),
        icon_anchor = (-120, 102),
        html='<div style="font-size: 20pt">Soletrader Map<div>',
        )
    ).add_to(f5_map)

display(f5_map)

f5_map.save('soletrader.html')